In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv


In [2]:
# Unicode Bidi override characters
PDF = chr(0x202C)
LRE = chr(0x202A)
RLE = chr(0x202B)
LRO = chr(0x202D)
RLO = chr(0x202E)

PDI = chr(0x2069)
LRI = chr(0x2066)
RLI = chr(0x2067)

class Swap():
    """Represents swapped elements in a string of text."""
    def __init__(self, one, two):
        self.one = one
        self.two = two
    
    def __repr__(self):
        return f"Swap({self.one}, {self.two})"

    def __eq__(self, other):
        return self.one == other.one and self.two == other.two

    def __hash__(self):
        return hash((self.one, self.two))

def some(*els):
    """Returns the arguments as a tuple with Nones removed."""
    return tuple(filter(None, tuple(els)))

def swaps(chars: str) -> set:
    """Generates all possible swaps for a string."""
    def pairs(chars, pre=(), suf=()):
        orders = set()
        for i in range(len(chars)-1):
            prefix = pre + tuple(chars[:i])
            suffix = suf + tuple(chars[i+2:])
            swap = Swap(chars[i+1], chars[i])
            pair = some(prefix, swap, suffix)
#             print(i)
#             print(pair)
#             print(some(prefix, swap))
            orders.add(pair)
            orders.update(pairs(suffix, pre=some(prefix, swap)))
            orders.update(pairs(some(prefix, swap), suf=suffix))
        return orders
    return pairs(chars) | {tuple(chars)}

def unswap(el: tuple) -> str:
    """Reverts a tuple of swaps to the original string."""
    if isinstance(el, str):
        return el
    elif isinstance(el, Swap):
        return unswap((el.two, el.one))
    else:
        res = ""
        for e in el:
            res += unswap(e)
        return res

def uniswap(els):
    res = ""
    for el in els:
        if isinstance(el, Swap):
            res += uniswap([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
        elif isinstance(el, str):
            res += el
        else:
            for subel in el:
                res += uniswap([subel])
    return res

def strings_to_file(file, string):
    with open(file, 'w') as f:
        for swap in swaps(string):
            uni = uniswap(swap)
            print(uni, file=f)

def print_strings(string):
    for swap in swaps(string):
        uni = uniswap(swap)
        print(uni)

In [3]:
# # Prove number of permutations = 2^|n|-1
# string = "A test string."
# assert(len(swaps(string))) == 2**(len(string)-1)

In [4]:
# Print strings
# print_strings("ennett")

In [5]:
unbidi_table = {
    chr(0x202a): "LRE",
    chr(0x202b): "RLE",
    chr(0x202c): "PDF",
    chr(0x202d): "LRO",
    chr(0x202e): "RLO",
    chr(0x2066): "LRI",
    chr(0x2067): "RLI",
    chr(0x2068): "FSI",
    chr(0x2069): "PDI",
}
def unbidi(code,unbidi_table) -> str:
    """
    Un-applies bidirectional control characters from a string.

    Args:
      code: The string to process.

    Returns:
      The unprocessed string with Bidi controls removed.
    """
    cnt = 0
    # Convert the string to bytes for character matching
    byte_code = code
#     print(byte_code)
    for char in unbidi_table.keys():
#         print(char)
        cnt += byte_code.count(char)
        # Replace characters using byte operations
        byte_code = byte_code.replace(char, unbidi_table[char])
    # Decode the modified bytes back to a string
#     unprocessed_code = byte_code.decode('utf-8')
    print(f"[i] Replaced {cnt} chars")
    return byte_code

In [6]:
# Save strings to file
# strings_to_file('reordering_example.txt', "This is a test.")

In [7]:
def execute_one_swap(string, seed):
    random.seed(seed)
    random_swap = random.choice(list(swaps(string)))
    print(random_swap)
    uni_text = uniswap(random_swap)
    return uni_text

In [8]:
# random.seed(16)
# random_swap = random.choice(list(swaps("Test")))
# print(random_swap)
# uni_text = uniswap(random_swap)
# print(uni_text)
# print(unbidi(uni_text,unbidi_table))
# random.seed(122)
# random_swap = random.choice(list(swaps("Test")))
# print(random_swap)
# uni_text = uniswap(random_swap)
# print(uni_text)
# print(unbidi(uni_text,unbidi_table))
# print(unswap(uni_text))

In [9]:
def unswap(text: str) -> str:
    """
    Reverses the effects of the uniswap function, removing Bidi control characters.

    Args:
      text: The string with Bidi control characters.

    Returns:
      The original string without Bidi control characters.
    """
    parts = []
    start = 0
    for i in range(len(text)):
        if text[i] in (LRO, RLO, LRI, RLI, PDI, PDF):
            parts.append(text[start:i])
            start = i + 1
    parts.append(text[start:])
    return ''.join(reversed(parts))


In [10]:
# # Read data from CSV
# data = pd.read_csv("/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv")
# # print(data)

# # Filter for watermarked "Generated text" with a limit of 1000 rows
# filtered_data = data[data["label"] == "watermarked"][["Generated Text"]]
# # Initialize empty list to store paraphrased text
# paraphrased_text = []

# # Loop through the data using tqdm for progress bar
# for text in tqdm(filtered_data["Generated Text"].tolist()):
#     # Perform round-trip translation and append to list
# #     print(text)
#     reordered = execute_one_swap(text,42)
# #     print(translated)
#     paraphrased_text.append(reordered)

# # Create a new dataframe with original and paraphrased text
# df = pd.DataFrame({"Original Text": filtered_data["Generated Text"], "Paraphrased Text": paraphrased_text})

# # Save dataframe as CSV file
# df.to_csv("bidi_reordered_attacked.csv", index=False)

# print("Reordering complete! Reordered data saved to bidi_reordered_attacked.csv")


In [11]:
import random

# Unicode Bidi override characters (already defined)

def split_swap_merge(sentence,seed):
    """Splits a sentence into chunks of 5 to 10 characters, performs 
    a random swap on each chunk, and merges them back into one sentence."""
    chunks = []
    start_index = 0
    random.seed(seed)
    while start_index < len(sentence):
        chunk_size = random.randint(5, 10)  # Random chunk size between 5 and 10
        end_index = min(start_index + chunk_size, len(sentence))
        chunk = sentence[start_index:end_index]
        chunks.append(chunk)
        start_index = end_index
    swapped_chunks = []
    for chunk in chunks:
#         print(len(chunk))
#         print(chunk)
        if len(chunk) > 1:
            swaps_set = swaps(chunk)  # Generate all possible swaps for the chunk
            random_swap = random.choice(list(swaps_set))  # Choose a random swap
            swapped_chunks.append(uniswap(random_swap))  # Apply the random swap
        else:
            swapped_chunks.append(chunk)
    return "".join(swapped_chunks)  # Merge chunks back into a sentence (no spaces)



In [12]:
# Read data from CSV
data = pd.read_csv("/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv")
# print(data)

# Filter for watermarked "Generated text" with a limit of 1000 rows
filtered_data = data[data["label"] == "watermarked"][["Generated Text"]]
# Initialize empty list to store paraphrased text
reordered_text = []

# Loop through the data using tqdm for progress bar
# count = 0
for text in tqdm(filtered_data["Generated Text"].tolist()):
    # Perform round-trip translation and append to list
    reordered = split_swap_merge(text,10)
#     count+=1
#     print(count)
#     print(reordered)
    reordered_text.append(reordered)

# Create a new dataframe with original and paraphrased text
df = pd.DataFrame({"Original Text": filtered_data["Generated Text"], "Reordered Text": reordered_text})

# Save dataframe as CSV file
df.to_csv("bidi_reordered_attacked.csv", index=False)

print("Reordering complete! Reordered data saved to bidi_reordered_attacked.csv")


100%|██████████| 1000/1000 [00:45<00:00, 22.14it/s]


Reordering complete! Reordered data saved to bidi_reordered_attacked.csv
